In [ ]:
%pip install datasets
%pip install transformers
%pip install transformers[torch]
%pip install evaluate
%pip install pandas==2.0.3

In [2]:
import sys
sys.path.append('/content/drive/MyDrive/Faks/research_uiktp')

In [3]:
import evaluate
import numpy as np
import pandas as pd
from datasets import DatasetDict
from make_dataset import split_dataset      # Module not found on google collab
from transformers import BertTokenizer, BertModel, AutoModelForSequenceClassification, TrainingArguments, Trainer

In [ ]:
#CSV_DATASET_PATH = "./process_data/process_data.csv"
CSV_DATASET_PATH = "/content/drive/MyDrive/Faks/research_uiktp/processed_data/processed_data.csv"   # directory not found on google collab
dataframe = pd.read_csv(CSV_DATASET_PATH)
print(dataframe)

In [ ]:
train_set, test_set, validation_set = split_dataset(dataframe, train_set_length=.8, test_set_length=.1, validation_set_length=.1, axis=0)
print(train_set)
print(test_set)
print(validation_set)

In [ ]:
dataset = DatasetDict(
    {
        "train":train_set,
        "test":test_set,
        "validation":validation_set
    }
)
print(dataset)

In [ ]:
tokenizer = BertTokenizer.from_pretrained("bert-base-cased")

def tokenize_function(jira_tasks):
    ret = tokenizer(jira_tasks["description"], padding="max-length", truncation=True)
    return ret

tokenized_datasets = dataset.map(tokenize_function, batched=True)
train_set = tokenized_datasets["train"]
test_set = tokenized_datasets["test"]
validation_set = tokenized_datasets["validation"]
print(train_set)
print(test_set)
print(validation_set)

In [ ]:
metric = evaluate.load("accuracy")

def compute_metrics(eval_pred):
    logits, labels = eval_pred
    predictions = np.argmax(logits, axis=-1)
    return metric.compute(predictions=predictions, references=labels)

In [ ]:
model = AutoModelForSequenceClassification.from_pretrained("bert-base-cased", num_labels=10)
training_args = TrainingArguments(output_dir="training_logs", evaluation_strategy="epoch")
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_set,
    eval_dataset=test_set,
    compute_metrics=compute_metrics
)
trainer.train()